In [1]:
import numpy as np
import pandas as pd
import gc

Load dataframes for inspection

In [2]:
amaz_df = pd.read_csv("MLOpsReviews/amazon_prime_titles.csv")
disn_df = pd.read_csv("MLOpsReviews/disney_plus_titles.csv")
hulu_df = pd.read_csv("MLOpsReviews/hulu_titles.csv")
netf_df = pd.read_csv("MLOpsReviews/netflix_titles.csv")

amaz_df.head()
# disn_df.head()
# hulu_df.head()
# netf_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


##### 1. Create id column using first letter of platform and contents of show_id column

In [3]:
datafs = (amaz_df, disn_df, hulu_df, netf_df)
for dfs in datafs:
    dfs.rename({"show_id": "id"}, axis=1, inplace=True)
else:
    del dfs, datafs # Delete loop variables, as they are no longer needed and use up resources otherwise
    gc.collect()
    amaz_df["id"] = "a" + amaz_df["id"]
    disn_df["id"] = "d" + disn_df["id"]
    hulu_df["id"] = "h" + hulu_df["id"]
    netf_df["id"] = "n" + netf_df["id"]

##### 2. Fill NaN values in rating columns with "G" rating, for "General for All Audiences"

In [4]:
rating_cols = (amaz_df.rating, disn_df.rating, hulu_df.rating, netf_df.rating)
for col in rating_cols:
    col.fillna(value="G", inplace=True)
else:
    del col, rating_cols # Delete loop variables, as they are no longer needed and use up resources otherwise
    gc.collect()

##### 3. Convert any dates to "YYYY-mm-dd" format

In [5]:
datafs = (amaz_df, disn_df, hulu_df, netf_df)
for dfs in datafs:
    dfs["date_added"] = pd.to_datetime(dfs["date_added"])
else:
    del dfs, datafs
    gc.collect()

##### 4. Turn all text fields to lowercase

In [6]:
amaz_df = amaz_df.applymap(lambda s: s.lower() if isinstance(s, str) else s)
disn_df = disn_df.applymap(lambda s: s.lower() if isinstance(s, str) else s)
hulu_df = hulu_df.applymap(lambda s: s.lower() if isinstance(s, str) else s)
netf_df = netf_df.applymap(lambda s: s.lower() if isinstance(s, str) else s)

##### 5. duration columns should be turned to 2 separate columns: duration_int & duration_type. The first must be of type int and be equal to the numeric part of the previous duration column, while the second must be a string equal to the non-numeric part of the duration column, indicating min or seasons, respectively]

In [7]:
datafs = (amaz_df, disn_df, hulu_df, netf_df)
for dfs in datafs:
    dfs[["duration_int", "duration_type"]] = dfs["duration"].str.split(" ", expand=True)
    dfs["duration_int"] = dfs["duration_int"].astype(pd.Int16Dtype())
    dfs.drop(["duration"], axis=1, inplace=True)
else:
    del dfs, datafs # Delete loop variables, as they are no longer needed and use up resources otherwise
    gc.collect()

In [8]:
amaz_df.head()
# disn_df.head()
# hulu_df.head()
# netf_df.head()

,id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,74,min
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",69,min
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,45,min


##### Put all cleaned dataframes in a single csv file

In [9]:
cleaned_datafs = pd.concat((amaz_df, disn_df, hulu_df, netf_df), ignore_index=True)
del amaz_df, disn_df, hulu_df, netf_df
gc.collect()

# Figure out platform based on substring in id
def check_platform(id_string: str) -> str:
    if id_string.startswith("a"):
        return "amazon"
    elif id_string.startswith("d"):
        return "disney"
    elif id_string.startswith("h"):
        return "hulu"
    elif id_string.startswith("n"):
        return "netflix"
    else:
        return np.NaN

if "platform" in cleaned_datafs.columns:
    cleaned_datafs.drop(["platform"], axis=1, inplace=True) # Drop column if already exists

cleaned_datafs.insert(loc=1, column="platform", value=cleaned_datafs["id"].map(check_platform))
cleaned_datafs.to_csv("cleaned_dbs.csv", index=False)
cleaned_datafs

,id,platform,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type
0,as1,amazon,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min
1,as2,amazon,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min
2,as3,amazon,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,74,min
3,as4,amazon,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",69,min
4,as5,amazon,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,45,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,ns8803,netflix,movie,zodiac,david fincher,"mark ruffalo, jake gyllenhaal, robert downey j...",united states,2019-11-20,2007,r,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",158,min
22994,ns8804,netflix,tv show,zombie dumb,NaN,NaN,NaN,2019-07-01,2018,tv-y7,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",2,seasons
22995,ns8805,netflix,movie,zombieland,ruben fleischer,"jesse eisenberg, woody harrelson, emma stone, ...",united states,2019-11-01,2009,r,"comedies, horror movies",looking to survive in a world taken over by zo...,88,min
22996,ns8806,netflix,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,pg,"children & family movies, comedies","dragged from civilian life, a former superhero...",88,min


##### Create platform column for rating csv files. First, join all files into 1 single dataframe

In [10]:
from pathlib import Path

# Create a glob string representing all csv files inside "ratings" directory
globbed_path = Path("./MLOpsReviews/ratings/").glob("*.csv")

# Join all files into a single pandas DataFrame object
ratings_df = pd.concat(map(pd.read_csv, globbed_path), ignore_index=True)
ratings_df.to_csv("raw_ratings.csv", index=False)

# Finally, we create the platform column based on movieId
if "platform" in ratings_df.columns:
    ratings_df.drop("platform", axis=1, inplace=True) # Drop platform column if already exists
ratings_df.insert(loc=len(ratings_df.columns), column="platform", value=ratings_df["movieId"].map(check_platform))
display(ratings_df.head(), ratings_df.shape)

,userId,rating,timestamp,movieId,platform
0,1,1.0,1425941529,as680,amazon
1,1,4.5,1425942435,ns2186,netflix
2,1,5.0,1425941523,hs2381,hulu
3,1,5.0,1425941546,ns3663,netflix
4,1,5.0,1425941556,as9500,amazon


(11024289, 5)

##### Getting the average rating of movies based on movieId column

In [11]:
mean_score = ratings_df.groupby("movieId")["rating"].mean().round(1)
processed_ratings = ratings_df.join(mean_score, on="movieId", rsuffix="_avg")
processed_ratings = processed_ratings.drop_duplicates(subset="movieId", keep="first")

del globbed_path, mean_score, ratings_df
gc.collect()

processed_ratings.rename(
    columns={
        "rating": "score",
        "rating_avg": "score_avg"
    },
    inplace=True
)
processed_ratings.to_csv("processed_ratings.csv", index=False)
display(processed_ratings.head(), processed_ratings.shape)

,userId,score,timestamp,movieId,platform,score_avg
0,1,1.0,1425941529,as680,amazon,3.5
1,1,4.5,1425942435,ns2186,netflix,3.6
2,1,5.0,1425941523,hs2381,hulu,3.6
3,1,5.0,1425941546,ns3663,netflix,3.5
4,1,5.0,1425941556,as9500,amazon,3.5


(22998, 6)

##### Merge both cleaned_datafs and processed_ratings DataFrame objects into a single csv file

In [12]:
complete_db = cleaned_datafs.merge(
    processed_ratings,
    how="outer",
    left_on="id",
    right_on="movieId",
    suffixes=(None, "_y"),
    validate="1:1"
)

if "platform_y" in complete_db.columns:
    complete_db.drop(["platform_y", "score", "userId", "timestamp", "movieId"], axis=1, inplace=True)
complete_db.to_csv("movies_db.csv", index=False)
display(complete_db.info(), complete_db.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 22997
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             22998 non-null  object        
 1   platform       22998 non-null  object        
 2   type           22998 non-null  object        
 3   title          22998 non-null  object        
 4   director       14739 non-null  object        
 5   cast           17677 non-null  object        
 6   country        11499 non-null  object        
 7   date_added     13444 non-null  datetime64[ns]
 8   release_year   22998 non-null  int64         
 9   rating         22998 non-null  object        
 10  listed_in      22998 non-null  object        
 11  description    22994 non-null  object        
 12  duration_int   22516 non-null  Int16         
 13  duration_type  22516 non-null  object        
 14  score_avg      22998 non-null  float64       
dtypes: Int16(1), dateti

None

,id,platform,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type,score_avg
0,as1,amazon,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min,3.5
1,as2,amazon,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min,3.5
2,as3,amazon,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,74,min,3.5
3,as4,amazon,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",69,min,3.5
4,as5,amazon,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,45,min,3.5
